<a href="https://colab.research.google.com/github/LashawnFofung/Python-Document-Preparation-and-Extraction/blob/main/Task_Extract_Key_Fields_from_the_Loan_Worksheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Key Fields

In this task, I will build a mini extraction pipeline for document automation.

**Data:** *Loan Worksheet PDF*

**Goal:** Pull out (extract) both plain text fields and structued table data using PyMuPDF.

# Identify Key Information from the Loan Worksheet PDF

I identified nine (9) important fields that should be extracted from this document:


1. Applicant Name: Identifies the customer
2. Loan Amount: Key financial indicator
2. Interest Rate: Useful for comparing lenders
3. Date Prepared: Date worksheet prepared
4. Loan Program:
5. Term/Due In:
6. Prepared By: Name of the lender
7. Underwriting Fee
8. Appraisal Fee
Closing/Escrow Fee
9. Lender's Title Fee


The nine selected fields above do not represent all of the important fields within the document.






In [13]:
#Install PyMuPDF
!pip install pymupdf

In [2]:
#Upload the LenderFeesWorksheetNew.pdf file
from google.colab import files
uploaded = files.upload()

Saving LenderFeesWorksheetNew.pdf to LenderFeesWorksheetNew (2).pdf


In [14]:
#Import PyMuPDF (fitz) and extract tables
import fitz

In [19]:
# This function opens a PDF, extracts all tables on every page
# Returns the data along with the bounding box (bbox) coordinates
# for the entire table and for each individual cell.


def extract_tables_with_bbox(pdf_path):
  doc = fitz.open(pdf_path)
  extracted_data = []

  for page_num in range(len(doc)):
    page = doc[page_num]

    #Extract General Text Blocks (for non-table fields)
    #'blocks' modegives text, bbox, and other properties
    text_blocks = page.get_text("blocks")
    for block in text_blocks:
        # block[4] is the text content, block[:4] is the bbox (x0, y0, x1, y1)
        text = block[4].strip()
        bbox = list(block[:4])


        # Only store non-empty text blocks
        if text:
            extracted_data.append({
                'type': 'text_block',
                'page': page_num,
                'text': text.replace('\n', ' '), #Replace newlines for easier searching
                'bbox': bbox
            })

        #Extract Tables (using existing logic, simplified for integration)
        tables_finder = page.find_tables()

        for i, table in enumerate(tables_finder.tables):
            table_bbox = list(table.bbox)


        #Iterate over cellls to get text and individual cell bboxes
        for row_num in range(table.row_count):
            row_data = []


            for col_num in range(table.col_count):
                cell_idx = row_num * table.col_count + col_num

                if cell_idx < len(table.cells) and table.cells[cell_idx] is not None:
                    cell_bbox = table.cells[cell_idx]
                    # Use get_text with the cell's bbox to extract content
                    cell_text = page.get_text("text", clip=cell_bbox).strip()

                    # Store as a structed fee item if it looks like a fee row
                    if row_num > 0 and col_num == 0 and cell_text: #Assuming first column is the fee name
                    # Ensure cell_idx + (table.col_count - 1) is a valid index before clipping
                        amount_cell_idx = cell_idx + (table.col_count - 1)
                        if amount_cell_idx < len(table.cells) and table.cells[amount_cell_idx] is not None:
                            # Extract from the last column (amount)
                            fee_amount = page.get_text("text", clip=table.cells[amount_cell_idx]).strip()

                            extracted_data.append({
                                'type': 'fee_item',
                                'page': page_num,
                                'fee_name': cell_text,
                                'bbox': list(cell_bbox) #Use the fee name's bbox for locating
                            })
                    row_data.append(cell_text)

                else:
                    row_data.append("")


            # Store the overall table structure
            extracted_data.append({
                'type': 'table_structure',
                'page': page_num,
                'table_index': i,
                'bbox': table_bbox,
                'data': table.cells #store raw cells or raw_data for debugging

            })

  doc.close()
  return extracted_data



In [20]:
# Call the function on the upload file
pdf_path = list(uploaded.keys())[0]

# Use the function name defined above
all_extracted_data = extract_tables_with_bbox(pdf_path)

print(f"Data extraction complete for: {pdf_path}")

Data extraction complete for: LenderFeesWorksheetNew (2).pdf


In [23]:
#Identify and Extract the Nine Key Fields

#The 9 key fields I identified
target_fields = [
    "Applicant Name:", "Loan Amount:", "Interest Rate:",
    "Date Prepared:", "Loan Program:", "Term/Due In:",
    "Prepared By:", "Underwriting Fee", "Appraisal Fee",
    "Closing/Escrow Fee", "Lender's Title Fee"
]

#Create a consolidated list for the final output
key_field_results = []

#Use a set to track which fields we've already found to avoid duplicates
found_fields = set()
print("## Extracted Key Fields and Bounding Boxes\n")
print("--------------------------------------------------")


#1. Search General Text Blocks (fr non-fee fields)
for item in all_extracted_data:
    if item['type'] == 'text_block':
        text = item['text']

        # Check against non-fee fields
        for field in target_fields[:7]: # Check first 7 (non-fee) fields
          if field in text and field not in found_fields:
              # Basic string processing to isolate the value
              value = text.split(field)[-1].strip()
              if value and value != field.strip():
                  extracted_text = f"{field} {value}"

              #if value is in a separate block or the field is the value itself
              else:
                extracted_text = text

              key_field_results.append({
                  "field": field.replace(':', ''),
                  "text": extracted_text,
                  "bbox": item['bbox']
              })
              print(f"**{field.replace(':', '')}**\nText: {extracted_text}\nBbox: {item['bbox']}\n")
              found_fields.add(field)


# 2. Search Fee Items (for fee fields)
print("## Extracted Fee Fields\n")
print("-----------------------------------------------------")

for item in all_extracted_data:
    if item['type'] == 'fee_item':
        fee_name = item['fee_name']

        # Check against fee fields
        for fee_field in target_fields[7:]: # Check last 4 (fee) fields


            # Use 'in' because the fee name might be descriptive (e.g., 'ABC Underwriting Fee')
            if fee_field in fee_name and fee_field not in found_fields:
                extracted_text = f"{fee_field}: {item['fee_amount']}"

                key_field_results.append({
                    "field": fee_field,
                    "text": extracted_text,
                    "bbox": item['bbox']
                })
                print(f"**{fee_field}**\nText: {extracted_text}\nBbox: {item['bbox']}\n")
                found_fields.add(fee_field)

print("-----------------------------------------------------")
print(f"Total unique fields found: {len(key_field_results)}")



## Extracted Key Fields and Bounding Boxes

--------------------------------------------------
**Date Prepared**
Text: Date Prepared: Loan Program: Prepared By:
Bbox: [39.599998474121094, 78.64180755615234, 463.07476806640625, 107.48875427246094]

**Loan Program**
Text: Loan Program: Prepared By:
Bbox: [39.599998474121094, 78.64180755615234, 463.07476806640625, 107.48875427246094]

**Prepared By**
Text: Applicants: Application No: Date Prepared: Loan Program: Prepared By:
Bbox: [39.599998474121094, 78.64180755615234, 463.07476806640625, 107.48875427246094]

**Loan Amount**
Text: Loan Amount: Interest Rate: Term/Due In:
Bbox: [39.599998474121094, 161.84454345703125, 460.9887390136719, 170.77496337890625]

**Interest Rate**
Text: Interest Rate: Term/Due In:
Bbox: [39.599998474121094, 161.84454345703125, 460.9887390136719, 170.77496337890625]

**Term/Due In**
Text: Total Loan Amount:   Interest Rate: Term/Due In:
Bbox: [39.599998474121094, 161.84454345703125, 460.9887390136719, 170.774963

In [24]:
# Identify and Extract the Nine Key Fields (Now addressing 11 fields)

target_fields = [
    "Applicant Name:", "Loan Amount:", "Interest Rate:",
    "Date Prepared:", "Loan Program:", "Term/Due In:",
    "Prepared By:", "Underwriting Fee", "Appraisal Fee",
    "Closing/Escrow Fee", "Lender's Title Fee"
]

key_field_results = []
# Use a set to track which fields we've already found to avoid duplicates
found_fields = set()

print("## Extracted Key Fields and Bounding Boxes\n")
print("-----------------------------------------------------")


# 1. Search General Text Blocks (for non-fee fields, more robustly)
# CORRECTION: Now checking all 11 fields against text blocks for redundancy,
# although fee items should ideally be found by type 'fee_item'.
for item in all_extracted_data:
    if item['type'] == 'text_block':
        text = item['text']

        # Check against ALL target fields in a single text block
        for field in target_fields:
            # Use 'in' for flexible matching and check if it hasn't been found already
            if field in text and field not in found_fields:

                # Basic string processing to isolate the value (works best for fields ending in ':')
                if ":" in field:
                    value = text.split(field)[-1].strip()
                    if value and value != field.strip():
                        extracted_text = f"{field} {value}"
                    else:
                        extracted_text = text
                else:
                    # For non-colon fields (often fees or headers), just use the whole text block
                    extracted_text = text

                # Check for duplicates again just before adding
                if field not in found_fields:
                    key_field_results.append({
                        "field": field.replace(':', ''),
                        "text": extracted_text,
                        "bbox": item['bbox']
                    })
                    print(f"**{field.replace(':', '')}**\nText: {extracted_text}\nBbox: {item['bbox']}\n")
                    found_fields.add(field)


# 2. Search Fee Items (for fee fields extracted directly from tables)
print("## Extracted Fee Fields (from table data)\n")
print("-----------------------------------------------------")

for item in all_extracted_data:
    if item['type'] == 'fee_item':
        fee_name = item['fee_name']

        # Check against fee fields (index 7 onwards)
        for fee_field in target_fields[7:]:

            # Use 'in' because the fee name might be descriptive (e.g., 'ABC Underwriting Fee')
            # Only add it if it hasn't already been picked up from a text_block (unlikely, but possible)
            if fee_field in fee_name and fee_field not in found_fields:
                extracted_text = f"{fee_field}: {item['fee_amount']}"

                key_field_results.append({
                    "field": fee_field,
                    "text": extracted_text,
                    "bbox": item['bbox']
                })
                print(f"**{fee_field}**\nText: {extracted_text}\nBbox: {item['bbox']}\n")
                found_fields.add(fee_field)

print("-----------------------------------------------------")
print(f"Total unique fields found: {len(key_field_results)}")

## Extracted Key Fields and Bounding Boxes

-----------------------------------------------------
**Date Prepared**
Text: Date Prepared: Loan Program: Prepared By:
Bbox: [39.599998474121094, 78.64180755615234, 463.07476806640625, 107.48875427246094]

**Loan Program**
Text: Loan Program: Prepared By:
Bbox: [39.599998474121094, 78.64180755615234, 463.07476806640625, 107.48875427246094]

**Prepared By**
Text: Applicants: Application No: Date Prepared: Loan Program: Prepared By:
Bbox: [39.599998474121094, 78.64180755615234, 463.07476806640625, 107.48875427246094]

**Loan Amount**
Text: Loan Amount: Interest Rate: Term/Due In:
Bbox: [39.599998474121094, 161.84454345703125, 460.9887390136719, 170.77496337890625]

**Interest Rate**
Text: Interest Rate: Term/Due In:
Bbox: [39.599998474121094, 161.84454345703125, 460.9887390136719, 170.77496337890625]

**Term/Due In**
Text: Total Loan Amount:   Interest Rate: Term/Due In:
Bbox: [39.599998474121094, 161.84454345703125, 460.9887390136719, 170.774